<a href="https://colab.research.google.com/github/HwangWonYoung/quant_project/blob/master/%08main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.0 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [ ]:
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from transformers import BertModel
from dataset import NasaDataset
from utils import model_inference
from trainer import train_model

In [ ]:
sys.path.append('/content/drive/MyDrive/KoBERT-Transformers-master')
from kobert_transformers.tokenization_kobert import KoBertTokenizer

In [ ]:
os.chdir('/content/drive/MyDrive/NASAData/')
dev = pd.read_csv('nasa23_c2_dat_dev.csv', sep='|')
submit = pd.read_csv('nasa23_c2_dat_submit.csv', sep='|')
nspsc = pd.read_csv('nasa23_c2_nspsc.csv', sep='|')

In [ ]:
# join data
data_df = pd.merge(dev, nspsc, how='left', on='goods_cd')
data_df = data_df.loc[data_df['goods_nm'].notna(), :]

In [ ]:
# split data
n_train = round(len(data_df) * 0.8)
train_df = data_df.iloc[:n_train, :]
val_df = data_df.iloc[n_train:, :]

NasaDataset 에서 false pair 까지 추출하도록 수정

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class NasaDatasetWithFalsePair(Dataset):
    def __init__(self, data_df, nspsc_df, tokenizer, false_pair_level, false_pair_min_num, n_seq=32):
        self.data_df = data_df
        self.nspsc_df = nspsc_df
        self.tokenizer = tokenizer
        self.false_pair_level = false_pair_level
        self.false_pair_min_num = false_pair_min_num
        self.pad_token_id = tokenizer.pad_token_id
        self.n_seq = n_seq

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, index):
        biz_desc_input = self.tokenizer(self.data_df.iloc[index, 0])
        n_biz_desc = len(biz_desc_input['input_ids'])
        biz_desc_input['input_ids'] = torch.tensor(
            biz_desc_input['input_ids'] + [self.pad_token_id] * (self.n_seq - n_biz_desc))
        biz_desc_input['attention_mask'] = torch.tensor(
            biz_desc_input['attention_mask'] + [0] * (self.n_seq - n_biz_desc))
        del biz_desc_input['token_type_ids']

        goods_nm_input = self.tokenizer(self.data_df.iloc[index, 4])
        n_goods_nm = len(goods_nm_input['input_ids'])
        goods_nm_input['input_ids'] = torch.tensor(
            goods_nm_input['input_ids'] + [self.pad_token_id] * (self.n_seq - n_goods_nm))
        goods_nm_input['attention_mask'] = torch.tensor(
            goods_nm_input['attention_mask'] + [0] * (self.n_seq - n_goods_nm))
        del goods_nm_input['token_type_ids']

        goods_cd = str(self.data_df.iloc[index, 1])

        index1 = (self.nspsc_df['goods_cd'].astype('str').str.slice(0,self.false_pair_level) == goods_cd[:self.false_pair_level])
        index2 = (self.nspsc_df['goods_cd'].astype('str') != goods_cd)
        nspsc_false_pair = self.nspsc_df[index1&index2]

        false_pair_inputs = get_tokenizing_result(nspsc_false_pair, self.tokenizer, self.pad_token_id, self.n_seq, self.false_pair_min_num)
        false_pair_inputs = false_pair_inputs[:self.false_pair_min_num]

        return (self.data_df.iloc[index, 0], biz_desc_input), (self.data_df.iloc[index, 4], goods_nm_input), (false_pair_inputs)

In [ ]:
train_dataset = NasaDatasetWithFalsePair(train_df, nspsc, tokenizer, 5, 10)

In [ ]:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
batch_iterator = iter(train_loader)

In [ ]:
next(batch_iterator)

RuntimeError: ignored

In [ ]:
index = 10
false_pair_level = 5

In [ ]:
goods_nm = str(data_df.iloc[index, 4])
goods_cd = str(data_df.iloc[index, 1])

In [ ]:
index1 = (nspsc['goods_cd'].astype('str').str.slice(0,5) == goods_cd[:5])
index2 = (nspsc['goods_cd'].astype('str') != goods_cd)
nspsc_false_pair = nspsc[index1&index2]

In [ ]:
def get_tokenizing_result(false_pair_df, tokenizer, pad_token_id, n_seq, min_num_result):

  tokenized_result = []

  for _, row in false_pair_df.iterrows():
    goods_nm_input = tokenizer(row['goods_nm'])
    n_goods_nm = len(goods_nm_input['input_ids'])
    goods_nm_input['input_ids'] = torch.tensor(
            goods_nm_input['input_ids'] + [pad_token_id] * (n_seq - n_goods_nm))
    goods_nm_input['attention_mask'] = torch.tensor(
            goods_nm_input['attention_mask'] + [0] * (n_seq - n_goods_nm))
    tokenized_result.append(goods_nm_input)


  if len(tokenized_result)<min_num_result:
    for i in range(0, min_num_result-len(tokenized_result)):
      goods_nm_input = {}
      goods_nm_input['input_ids'] = torch.tensor([pad_token_id] * (n_seq))
      goods_nm_input['attention_mask'] = torch.tensor([0] * (n_seq ))
      tokenized_result.append(goods_nm_input)

  return tokenized_result

In [ ]:
# get_tokenizing_result(nspsc_false_pair, tokenizer, pad_token_id=1, n_seq=32)

In [ ]:
nspsc_false_pair

,goods_cd,goods_grp_gb,goods_nm,alias_nm
90730,7012150000,Class,낙농업,NaN
90731,7012150100,Commodity,낙농가축 관리,NaN
90732,7012150200,Commodity,낙농업개발,NaN
90733,7012150300,Commodity,낙농기술,NaN
90734,7012150400,Commodity,낙농연구서비스,NaN
90735,7012150500,Commodity,낙농가내가공,NaN
90737,7012160100,Commodity,가축번식,NaN
90738,7012160200,Commodity,가축유전학관련 서비스,NaN
90739,7012160300,Commodity,양잠,NaN
90740,7012160400,Commodity,축산,NaN


In [ ]:
# tokenizer init
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
# import pretrained model
model = BertModel.from_pretrained('monologg/kobert').cuda()

In [ ]:
# set params trainable
for param in model.parameters():
    param.requires_grad = True

In [ ]:
# Create the dataset
train_dataset = NasaDataset(train_df, tokenizer)
val_dataset = NasaDataset(val_df, tokenizer)

In [ ]:
# Create data loaders
batch_size = 64
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# train model
model, loss_output, val_loss_output = train_model(model=model, train_loader=train_loader, val_loader=val_loader,
                                                  num_epochs=2, margin=0.1)
print(f'train_loss: {loss_output}')
print(f'val_loss: {val_loss_output}')

 50%|█████     | 1/2 [01:44<01:44, 104.54s/it]

Epoch [1/2] - Train Loss: 0.2340 - Val Loss: 0.1322


100%|██████████| 2/2 [03:29<00:00, 104.68s/it]

Epoch [2/2] - Train Loss: 0.1552 - Val Loss: 0.1130
train_loss: [0.23398310328392605, 0.15522808743346678]
val_loss: [0.13219099158519193, 0.11299766413867474]


In [ ]:
class NasaDataset(Dataset):
    def __init__(self, data_df, tokenizer, n_seq=32):
        self.data_df = data_df
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id
        self.n_seq = n_seq

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, index):
        biz_desc_input = self.tokenizer(self.data_df.iloc[index, 0])
        n_biz_desc = len(biz_desc_input['input_ids'])
        biz_desc_input['input_ids'] = torch.tensor(
            biz_desc_input['input_ids'] + [self.pad_token_id] * (self.n_seq - n_biz_desc))
        biz_desc_input['attention_mask'] = torch.tensor(
            biz_desc_input['attention_mask'] + [0] * (self.n_seq - n_biz_desc))
        del biz_desc_input['token_type_ids']

        goods_nm_input = self.tokenizer(self.data_df.iloc[index, 4])
        n_goods_nm = len(goods_nm_input['input_ids'])
        goods_nm_input['input_ids'] = torch.tensor(
            goods_nm_input['input_ids'] + [self.pad_token_id] * (self.n_seq - n_goods_nm))
        goods_nm_input['attention_mask'] = torch.tensor(
            goods_nm_input['attention_mask'] + [0] * (self.n_seq - n_goods_nm))
        del goods_nm_input['token_type_ids']

        goods_cd = self.data_df.iloc[index, 1]

        return (self.data_df.iloc[index, 0], biz_desc_input), (self.data_df.iloc[index, 4], goods_nm_input), str(goods_cd)

In [ ]:
# model=model
# train_loader=train_loader
# val_loader=val_loader
num_epochs=2
margin=0.1
use_cuda=True

In [ ]:
# Initialize model and other parameters
learning_rate = 0.0001

# Initialize the model and optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CosineEmbeddingLoss(margin=margin)
if use_cuda:
    model = model.to('cuda')

In [ ]:
batch_iterator = iter(train_loader)

In [ ]:
((_, biz_desc), (_, goods_nm), goods_cd) = next(batch_iterator)

In [ ]:
# goods_cd = str(self.data_df.iloc[index, 1])

index1 = (self.nspsc_df['goods_cd'].astype('str').str.slice(0,self.false_pair_level) == goods_cd[:self.false_pair_level])
index2 = (self.nspsc_df['goods_cd'].astype('str') != goods_cd)
nspsc_false_pair = self.nspsc_df[index1&index2]

false_pair_inputs = get_tokenizing_result(nspsc_false_pair, self.tokenizer, self.pad_token_id, self.n_seq, self.false_pair_min_num)
false_pair_inputs = false_pair_inputs[:self.false_pair_min_num]

In [ ]:
def get_tokenizing_result(false_pair_df, tokenizer, pad_token_id, n_seq):

  input_ids = []
  attention_mask = []

  for _, row in false_pair_df.iterrows():
    goods_nm_input = tokenizer(row['goods_nm'])
    n_goods_nm = len(goods_nm_input['input_ids'])
    goods_nm_input['input_ids'] = torch.tensor(
            goods_nm_input['input_ids'] + [pad_token_id] * (n_seq - n_goods_nm))
    goods_nm_input['attention_mask'] = torch.tensor(
            goods_nm_input['attention_mask'] + [0] * (n_seq - n_goods_nm))
    input_ids.append(goods_nm_input['input_ids'])
    attention_mask.append(goods_nm_input['attention_mask'])

  input_ids_tensors = torch.stack(input_ids, dim=0)
  attention_mask_tensors = torch.stack(attention_mask, dim=0)

  return input_ids_tensors, attention_mask_tensors

In [ ]:
def get_nspsc_false_pairs(nspsc_df, goods_cd_list, false_pair_level, min_false_pair_num, tokenizer, pad_token_id=1, n_seq=32):

  output = {}

  for goods_cd in goods_cd_list:
    index1 = (nspsc_df['goods_cd'].astype('str').str.slice(0,false_pair_level) == goods_cd[:false_pair_level])
    index2 = (nspsc_df['goods_cd'].astype('str') != goods_cd)
    nspsc_false_pair = nspsc_df[index1&index2]

    input_ids = []
    attention_mask = []

    if len(nspsc_false_pair)<min_false_pair_num:
      for i in range(min_false_pair_num-len(nspsc_false_pair)):
        input_ids.append(torch.tensor([pad_token_id] * (n_seq)))
        attention_mask.append(torch.tensor(([0] * n_seq)))

    for _, row in nspsc_false_pair.iterrows():
      goods_nm_input = tokenizer(row['goods_nm'])
      n_goods_nm = len(goods_nm_input['input_ids'])
      goods_nm_input['input_ids'] = torch.tensor(
              goods_nm_input['input_ids'] + [pad_token_id] * (n_seq - n_goods_nm))
      goods_nm_input['attention_mask'] = torch.tensor(
              goods_nm_input['attention_mask'] + [0] * (n_seq - n_goods_nm))
      input_ids.append(goods_nm_input['input_ids'])
      attention_mask.append(goods_nm_input['attention_mask'])

    input_ids_tensors = torch.stack(input_ids, dim=0)
    attention_mask_tensors = torch.stack(attention_mask, dim=0)

    output[goods_cd] = input_ids_tensors, attention_mask_tensors

  return output

In [ ]:
a= get_nspsc_false_pairs(nspsc_false_pair, ('1214190200','7012160000'), 5, 10, tokenizer, pad_token_id=1, n_seq=32)

In [ ]:
a.keys()

dict_keys(['1214190200', '7012160000'])

In [ ]:
a['7012160000']

(tensor([[   2, 1404, 5734, 6873,    3,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [   2, 1404, 5734, 5330, 7463, 1088,    3,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [   2, 1404, 5734, 6873, 5362,    3,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [   2, 1404, 5734, 5575,    3,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1],
         [   2, 1404, 5734, 6929, 6555,    3,    1,    1,    1,    1,    1,    1,
             1,    1,    1

In [ ]:
# model(input_ids=a['7012160000'][0].to('cuda'), attention_mask=a['7012160000'][1].to('cuda'))

In [ ]:
if use_cuda:
    biz_desc['input_ids'] = biz_desc['input_ids'].to('cuda')
    biz_desc['attention_mask'] = biz_desc['attention_mask'].to('cuda')
    goods_nm['input_ids'] = goods_nm['input_ids'].to('cuda')
    goods_nm['attention_mask'] = goods_nm['attention_mask'].to('cuda')

In [ ]:
biz_output = model(input_ids=biz_desc['input_ids'], attention_mask=biz_desc['attention_mask'])
biz_output = biz_output['last_hidden_state'][:, 0, :]  # batch_size * 768
goods_output = model(input_ids=goods_nm['input_ids'], attention_mask=goods_nm['attention_mask'])
goods_output = goods_output['last_hidden_state'][:, 0, :]  # batch_size * 768

In [ ]:
false_pair = torch.randint(biz_output.size(0), (2, biz_output.size(0)))

In [ ]:
false_pair

tensor([[54, 57, 38, 32, 42, 40,  2, 29, 33,  1, 57, 26, 62, 61, 29, 55, 58, 33,
         35, 41,  9, 49, 30, 61, 58, 33, 21,  7, 48, 63, 62, 52, 21,  7, 33, 27,
          3,  7, 55, 42, 32, 34, 46, 33, 59, 56, 25, 28, 55, 35, 40, 41, 48, 43,
         34, 39, 59, 23, 19,  1,  2, 20, 20, 13],
        [ 7, 24, 19, 29,  4, 44,  0,  9, 20, 58, 52, 62, 29,  4,  4, 26, 21, 22,
         60, 42, 10, 37, 61, 21, 37, 45, 33,  1, 51, 31, 53, 29, 48, 36, 40, 50,
         10, 29, 25,  8, 10, 31,  1, 63, 18, 53, 38, 63,  3, 56,  9, 17, 16,  8,
         13, 62, 41, 13,  9, 10, 35, 51, 14, 10]])

In [ ]:
false_pair = false_pair[:, false_pair[0] != false_pair[1]]

In [ ]:
false_pair

tensor([[54, 57, 38, 32, 42, 40,  2, 29, 33,  1, 57, 26, 62, 61, 29, 55, 58, 33,
         35, 41,  9, 49, 30, 61, 58, 33, 21,  7, 48, 63, 62, 52, 21,  7, 33, 27,
          3,  7, 55, 42, 32, 34, 46, 33, 59, 56, 25, 28, 55, 35, 40, 41, 48, 43,
         34, 39, 59, 23, 19,  1,  2, 20, 20, 13],
        [ 7, 24, 19, 29,  4, 44,  0,  9, 20, 58, 52, 62, 29,  4,  4, 26, 21, 22,
         60, 42, 10, 37, 61, 21, 37, 45, 33,  1, 51, 31, 53, 29, 48, 36, 40, 50,
         10, 29, 25,  8, 10, 31,  1, 63, 18, 53, 38, 63,  3, 56,  9, 17, 16,  8,
         13, 62, 41, 13,  9, 10, 35, 51, 14, 10]])

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs, use_cuda=True, val_interval=1, margin=0.1):
    # Initialize model and other parameters
    learning_rate = 0.0001

    # Initialize the model and optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CosineEmbeddingLoss(margin=margin)
    if use_cuda:
        model = model.to('cuda')

    loss_output = []
    val_loss_output = []
    for epoch in tqdm(range(num_epochs)):
        model.train()
        total_loss = 0.0
        for batch_idx, ((_, biz_desc), (_, goods_nm)) in enumerate(train_loader):
            # batch_idx, ((biz_str, biz_desc), (goods_str, goods_nm)) = next(enumerate(train_loader))
            if use_cuda:
                biz_desc['input_ids'] = biz_desc['input_ids'].to('cuda')
                biz_desc['attention_mask'] = biz_desc['attention_mask'].to('cuda')
                goods_nm['input_ids'] = goods_nm['input_ids'].to('cuda')
                goods_nm['attention_mask'] = goods_nm['attention_mask'].to('cuda')

            optimizer.zero_grad()
            biz_output = model(input_ids=biz_desc['input_ids'], attention_mask=biz_desc['attention_mask'])
            biz_output = biz_output['last_hidden_state'][:, 0, :]  # batch_size * 768
            goods_output = model(input_ids=goods_nm['input_ids'], attention_mask=goods_nm['attention_mask'])
            goods_output = goods_output['last_hidden_state'][:, 0, :]  # batch_size * 768

            false_pair = torch.randint(biz_output.size(0), (2, biz_output.size(0)))
            false_pair = false_pair[:, false_pair[0] != false_pair[1]]
            if use_cuda:
                false_pair = false_pair.to('cuda')

            x1 = torch.cat([biz_output, biz_output[false_pair[0]]])
            x2 = torch.cat([goods_output, goods_output[false_pair[1]]])
            y = torch.cat([torch.tensor([1] * biz_output.size(0)), torch.tensor([-1] * false_pair[1].size(0))])
            if use_cuda:
                y = y.to('cuda')

            loss = criterion(x1, x2, y)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_loss = (total_loss / len(train_loader))
        loss_output.append(avg_loss)

        # Validation loop
        if epoch % val_interval == 0:
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for batch_idx, ((_, biz_desc), (_, goods_nm)) in enumerate(val_loader):
                    if use_cuda:
                        biz_desc['input_ids'] = biz_desc['input_ids'].to('cuda')
                        biz_desc['attention_mask'] = biz_desc['attention_mask'].to('cuda')
                        goods_nm['input_ids'] = goods_nm['input_ids'].to('cuda')
                        goods_nm['attention_mask'] = goods_nm['attention_mask'].to('cuda')

                    biz_output = model(input_ids=biz_desc['input_ids'], attention_mask=biz_desc['attention_mask'])
                    biz_output = biz_output['last_hidden_state'][:, 0, :]  # batch_size * 768
                    goods_output = model(input_ids=goods_nm['input_ids'], attention_mask=goods_nm['attention_mask'])
                    goods_output = goods_output['last_hidden_state'][:, 0, :]  # batch_size * 768

                    false_pair = torch.randint(biz_output.size(0), (2, biz_output.size(0)))
                    false_pair = false_pair[:, false_pair[0] != false_pair[1]]
                    if use_cuda:
                        false_pair = false_pair.to('cuda')

                    x1 = torch.cat([biz_output, biz_output[false_pair[0]]])
                    x2 = torch.cat([goods_output, goods_output[false_pair[1]]])
                    y = torch.cat([torch.tensor([1] * biz_output.size(0)), torch.tensor([-1] * false_pair[1].size(0))])
                    if use_cuda:
                        y = y.to('cuda')

                    loss = criterion(x1, x2, y)
                    val_loss += loss.item()

            avg_val_loss = (val_loss / len(val_loader))
            val_loss_output.append(avg_val_loss)
            print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {avg_loss:.4f} - Val Loss: {avg_val_loss:.4f}")

    return model, loss_output, val_loss_output